In [1]:
# macro
fileName = r'/home/liang.liang/code/Tools/python/pycharts/2019CW14.4.xlsx'
reportName = '20190405.html'
CWsMonitor = 14

TotalFeatures = 1026


In [2]:
# import lib

from pyecharts.engine import create_default_environment
from pyecharts import Page
from pyecharts import Bar, Line, Grid
from pyecharts import Liquid

import xlrd

In [3]:
data = xlrd.open_workbook(fileName)

table = data.sheets()[0]


Data =  {"KeyID":table.col_values(2,1),
         "funcGroup":table.col_values(4,1),
         "status":table.col_values(5,1),
         "group":table.col_values(6,1),
         "targetCW":table.col_values(7,1)

        }


















In [4]:
# define calss for project management

In [5]:
class Features:
    """it's 37w project"""
    __group  = ("Audio","Media","Radio","Online","HMI","SDS","AndroidApp","AndroidOS"
                   ,"Update","Navi","System","CarConnect","DVR","PCC")    
    
    __status = ("New","In Analysis","Pending","In Implementation","In Testing","Released")
    
    

    
    def __init__(self,source,export):
        self.__source = source
        self.__export = export
        
        self.__CWs = []
        for i in range(1,(CWsMonitor+1)):
            self.__CWs.append("CW%02d" %i)
            
            
        self.__featureLen = len(Data["KeyID"])
        self.__autowidth = 1200

        self.__countReleased = Data["status"].count("Released")

        
        
        
        self.__groupBar = {}
        for i in self.__group:
            self.__groupBar["%s" %i] =  {"New":0,"In Analysis":0,"Pending":0,"In Implementation":0,"In Testing":0,"Released":0}    
        
        for i in range(self.__featureLen):
            self.__groupBar[Data["group"][i]][Data["status"][i]] += 1
        
        
        self.__delay = {}
        for i in self.__status:
            self.__delay["%s" %i] = [0]*len(self.__group)
        
        for i in self.__status:
            groupCount = 0
            for j in self.__group:
                self.__delay[i][groupCount] = self.__groupBar[j][i]
                groupCount += 1
    
    
        self.__cwFeatures ={}
        for i in self.__status:
            self.__cwFeatures["%s" %i]=[0]*len(self.__CWs)
            
            
        for st in self.__status:
            for cw in range(0,CWsMonitor):
                cw1 = "%02d" % (cw+1)
                for i in range(0,self.__featureLen):
                    if(Data["status"][i] == st and Data["targetCW"][i] == cw1 ):
                        self.__cwFeatures[st][cw] += 1
    
    
        



    
        
    def GetExportFileName(self):
        return self.__export
        
        
    def ProjectFinishRate(self):
        liquid = Liquid(title="37W Project", title_text_size = 20
              ,subtitle = "Features Completion Rate")
        liquid.add("Liquid", [self.__countReleased/TotalFeatures],width = self.__autowidth/2)
        
        return liquid

    def GroupStatus(self):
        bar = Bar(title="Feature Status", title_text_size = 20
          ,subtitle ="Current status for each group"
          ,width = self.__autowidth)

        for i in self.__status:
            bar.add(i,self.__group, self.__delay[i] 
                ,xaxis_rotate=30,yaxis_rotate=0,xaxis_interval= 0
                ,is_label_show=False,yaxis_pos = "left"
                ,xaxis_type = 'category',is_stack = True,is_convert = False,yaxis_label_textsize = 12
                ,yaxis_name_gap = 5
                ,is_toolbox_show = True,is_more_utils=True
                ,yaxis_interval = 0,yaxis_margin= 0
                ,legend_pos="30%"
                   )
    
    
        return bar
    
    def CWStatus(self):
        barCW = Bar(title="Feature Status", title_text_size = 20
          ,subtitle ="Total items:" +str(self.__featureLen)
          ,width = self.__autowidth)

        for i in self.__status:
            barCW.add(i, self.__CWs, self.__cwFeatures[i]
            ,xaxis_rotate=0,yaxis_rotate=0,xaxis_interval= 0
            ,is_label_show=False,yaxis_pos = "left"
            ,xaxis_type = 'category',is_stack = True,is_convert = True,yaxis_label_textsize = 12
            ,yaxis_name_gap = 5
            ,is_toolbox_show = True,is_more_utils=True
            ,yaxis_interval = 0,yaxis_margin= 0
            ,legend_pos="30%"
           )
        
        return barCW
    
    
        


In [6]:
features = Features(fileName,reportName)

In [7]:
#coding=utf-8
from __future__ import unicode_literals


grid1 = Grid()


grid1.add(features.ProjectFinishRate(), grid_left=  "10%")


grid2 = Grid()
grid2.add(features.GroupStatus(),grid_left=  "10%")


grid3 = Grid()
#grid3.add(barCW,grid_left=  "10%")

grid3.add(features.CWStatus(),grid_left=  "10%")



page = Page()         # step 1

page.add(grid1)
page.add(grid2)
page.add(grid3)


page.render(path = features.GetExportFileName())

